In [1]:
from MBDOE import *
from generalize_functions import CEOSModels


[    0.00] Initializing mpi-sppy


## Choose experiments and model

In [2]:
model_option = "PR"
if_poly = False

if model_option=="PR":
    if not if_poly:
        from hfc32_emimtf2n_PR import configuration
    else:
        from hfc32_emimtf2n_PR_polynomial import configuration
        
elif model_option=="SRK":
    if not if_poly:
        from hfc32_emimtf2n_SRK import configuration
    else:
        from hfc32_emimtf2n_SRK_polynomial import configuration
        

In [3]:
# Experiments 
data_file = './emimtf2n/R32/Final_Results/MBDoE/r32_emimtf2n_subset.csv'

path = './emimtf2n/R32/Final_Results/MBDoE/Params/'

# Decide on model 
#para_file = path + 'PR_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'PR_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_1param_Opt2_GF.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'PR_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'PR_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'PR_params_noTdep.csv'
#PR_type_opt = "No"

#para_file = path + 'PR_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

# important model
para_file = path + 'PR_params_quadTdep.csv'
PR_type_opt = "Quadratic"

#para_file = path + 'SRK_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'SRK_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'SRK_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'SRK_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'SRK_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'SRK_params_noTdep.csv'
#PR_type_opt = "No"


#para_file = path + 'SRK_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

#para_file = path + 'SRK_params_quadTdep.csv'
#PR_type_opt = "Quadratic"



model_creation = CEOSModels(para_file, configuration, 
                         comp_1= "R32", comp_2 = "emimTf2N", 
                         x_comp_1="x_R32", x_comp_2="x_emimTf2N",  Model_type = model_option, Tdep_type=PR_type_opt)

## Run pyomo.doe

In [ ]:
data_exp = pd.read_csv(data_file)
print(data_exp)
print(data_exp.iloc[1])

In [ ]:
print(model_creation.param_name_dict)

In [ ]:
mbdoe_obj = MBDOE(data_file, model_creation)

exp_set = range(len(data_exp))
#exp_set = range(12)
#exp_set = [0,1,2,3]

#exp_set = 0
# init_T: 283-348; 323 is good; init_pressure: low mole fraction use 150000; high 399800; x_option: 0.4-0.6;0.3-0.55;
# SRK: init_temp_option=323, init_pressure_option= 399800, init_x_c1_option = 0.5

total_fim = mbdoe_obj.sumDOE(exp_set, scale_opt=True, record_name="SRK_polyTdep", 
                             init_temp_option=343, 
                             init_pressure_option= 150000,
                             init_x_c1_option = 0.4,
                             poly_option=True)
#total_fim = mbdoe_obj.doe(exp_set, scale=True)

In [ ]:
print(total_fim)
print(np.linalg.det(total_fim))
print(np.trace(total_fim))
print(np.linalg.eigvals(total_fim))

## Heatmap

In [4]:
# read prior

f = open('emimtf2n_FIM_info/PR_quadTdep.json')

data = json.load(f)

prior_info = data[0]['Total']
#print(prior_info)

for i in range(6):
    for j in range(6):
        prior_info[i][j] *= 10**(-10)

print(prior_info)
print(np.linalg.det(prior_info))
print(np.log10(np.linalg.det(prior_info)))

[[4557.535530992685, -1079.6186966711327, 8344.529017504288, -798.9512603441533, 3003.549094513206, -529.4333373880846], [-1079.6186966711327, 388.88133653439434, -1901.1816632190887, 279.054860245738, -657.7442271879426, 178.98151313955444], [8344.529017504288, -1901.1816632190887, 15359.201236486178, -1413.4663295880396, 5557.3688924934195, -941.178913439035], [-798.9512603441533, 279.054860245738, -1413.4663295880396, 200.80609121460168, -491.37691190048275, 129.18219737542094], [3003.549094513206, -657.7442271879426, 5557.3688924934195, -491.37691190048275, 2021.1335904074926, -328.8302607352068], [-529.4333373880846, 178.98151313955444, -941.178913439035, 129.18219737542094, -328.8302607352068, 83.37442064565272]]
11.49965617183968
1.0606848555792616


In [ ]:
#data_exp = pd.read_csv(data_file)
#print(data_exp)

In [5]:
# model list 
#T_range = [273, 360]
#x_range = [0.1, 0.9]

T_range = [273, 283, 293, 303, 313, 323, 333, 343, 353, 363, 373, 383, 393, 400]
x_range = [0.1,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

data_exp = {'pressure': 549300}

fixed_model_set = []

for i in range(len(T_range)):
    for j in range(len(x_range)):
        data_exp['temperature'] = T_range[i]
        data_exp['x_R32'] = x_range[j]
        data_exp['x_emimTf2N'] = 1-x_range[j]
    
        print(data_exp)
        
        model = model_creation.create_model(data_exp)
        
        print(model.fs.state_block.pressure.value)
        
        fixed_model_set.append(model)

{'pressure': 549300, 'temperature': 273, 'x_R32': 0.1, 'x_emimTf2N': 0.9}
2022-10-20 21:03:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.2, 'x_emimTf2N': 0.8}
2022-10-20 21:03:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.3, 'x_emimTf2N': 0.7}
2022-10-20 21:03:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.4, 'x_emimTf2N': 0.6}
2022-10-20 21:03:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.5, 'x_emimTf2N': 0.5}
2022-10-20 21:03:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Foun

399300
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.5, 'x_emimTf2N': 0.5}
2022-10-20 21:03:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.6, 'x_emimTf2N': 0.4}
2022-10-20 21:03:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.7, 'x_emimTf2N': 0.30000000000000004}
2022-10-20 21:03:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.8, 'x_emimTf2N': 0.19999999999999996}
2022-10-20 21:03:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 313, 'x_R32': 0.9, 'x_emimTf2N': 0.09999999999999998}
2022-10-20 21:03:39 [INFO] idaes.init.fs.state_block: Property 

399300
{'pressure': 549300, 'temperature': 353, 'x_R32': 0.9, 'x_emimTf2N': 0.09999999999999998}
2022-10-20 21:03:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.1, 'x_emimTf2N': 0.9}
2022-10-20 21:03:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.2, 'x_emimTf2N': 0.8}
2022-10-20 21:03:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.3, 'x_emimTf2N': 0.7}
2022-10-20 21:03:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 363, 'x_R32': 0.4, 'x_emimTf2N': 0.6}
2022-10-20 21:03:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal 

399300
{'pressure': 549300, 'temperature': 400, 'x_R32': 0.4, 'x_emimTf2N': 0.6}
2022-10-20 21:04:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 400, 'x_R32': 0.5, 'x_emimTf2N': 0.5}
2022-10-20 21:04:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 400, 'x_R32': 0.6, 'x_emimTf2N': 0.4}
2022-10-20 21:04:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 400, 'x_R32': 0.7, 'x_emimTf2N': 0.30000000000000004}
2022-10-20 21:04:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
399300
{'pressure': 549300, 'temperature': 400, 'x_R32': 0.8, 'x_emimTf2N': 0.19999999999999996}
2022-10-20 21:04:20 [INFO] idaes.init.fs.state_block: Property package initiali

In [6]:
design_range_values = [T_range, x_range]

In [7]:
mbdoe_obj = MBDOE(data_file, model_creation)

all_fim = mbdoe_obj.run_grid_search(design_range_values, fixed_models=fixed_model_set, 
                                    prior_FIM = prior_info, 
                                    scale_opt=True,
                                    scale_pressure=0.00001,
                                   store_name="PR_quad_543900_scaledBar")
                                    #, record_name="SRK_polyTdep", 
                                    # init_temp_option=343, 
                                    # init_pressure_option= 150000,
                                    # init_x_c1_option = 0.4,
                                    # poly_option=True)



parameter set: {'fs.properties.PR_kappa_A["R32", "emimTf2N"]': 0.3212980565927437, 'fs.properties.PR_kappa_A["emimTf2N", "R32"]': 0.8429816989080334, 'fs.properties.PR_kappa_B["R32", "emimTf2N"]': -0.5532108602621684, 'fs.properties.PR_kappa_B["emimTf2N", "R32"]': -0.6100855368792957, 'fs.properties.PR_kappa_C["R32", "emimTf2N"]': 0.1862684932461356, 'fs.properties.PR_kappa_C["emimTf2N", "R32"]': -0.3935451934926509}
All measurements are flattened.
Flatten measurement name: ['fs.state_block.pressure']
126  design vectors will be searched.
=======This is the  1 th iteration=======
Design variable values of this iteration: {'fs.F101.inlet.temperature': {0: 1}, 'fs.F101.inlet.pressure': {0: 1}, 'fs.F101.inlet.mole_frac_comp[0,"R32"]': {0: 1}, "fs.F101.inlet.mole_frac_comp[0,'emimTf2N']": {0: 1}}
Sensitivity information is scaled by its corresponding parameter nominal value.
Sensitivity information is scaled by constant  1e-05  times itself.
After practice: {'fs.state_block.pressure': [0]}

pressure: 338799.20797033637
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.4836273193359375e-06
Solve time with direct kaug mode [s]: 0.4706449508666992
Total wall clock time [s]: 0.47185850143432617
Existed information has been added.
FIM: [[ 4630.95094836 -1099.77333876  8460.28350629  -812.30543118
   3039.23974923  -537.31994175]
 [-1099.77333876   394.41436505 -1932.95959727   282.72096395
   -667.54233769   181.1466127 ]
 [ 8460.28350629 -1932.95959727 15541.71197    -1434.52192354
   5613.64254235  -953.61376553]
 [ -812.30543118   282.72096395 -1434.52192354   203.23519787
   -497.8689965    130.61676066]
 [ 3039.23974923  -667.54233769  5613.64254235  -497.8689965
   2038.48448045  -332.66430654]
 [ -537.31994175   181.1466127   -953.61376553   130.61676066
   -332.66430654    84.22163409]]
Trace: 22893.0185958057
Determinant: 17.23101330904678
Condition number: 260804823.37121418
Minimal eigen value: 8.671973116500241e-05
Eigen values: [2.26169242e+04 

pressure: 726075.3131859299
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 4.5299530029296875e-06
Solve time with direct kaug mode [s]: 0.47503137588500977
Total wall clock time [s]: 0.4756810665130615
Existed information has been added.
FIM: [[ 4588.93994595 -1123.54960699  8394.04665195  -828.06085188
   3018.81755228  -546.62561108]
 [-1123.54960699   450.33527174 -1970.45073769   319.77559512
   -679.1029199    203.03138749]
 [ 8394.04665195 -1970.45073769 15437.27930686 -1459.36554836
   5581.44378487  -968.28722687]
 [ -828.06085188   319.77559512 -1459.36554836   227.78854959
   -505.52965347   145.11817534]
 [ 3018.81755228  -679.1029199   5581.44378487  -505.52965347
   2028.55693527  -337.18894231]
 [ -546.62561108   203.03138749  -968.28722687   145.11817534
   -337.18894231    92.78629076]]
Trace: 22825.68630017333
Determinant: 48.25011786726154
Condition number: 155230821.57424235
Minimal eigen value: 0.0001448342964430667
Eigen values: [2.24827468e+04

pressure: 348954.593239589
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 4.0531158447265625e-06
Solve time with direct kaug mode [s]: 0.46256303787231445
Total wall clock time [s]: 0.4631955623626709
Existed information has been added.
FIM: [[ 4640.74144776 -1095.48072664  8480.52626497  -809.8455958
   3047.01734202  -536.10254139]
 [-1095.48072664   391.90520796 -1927.10761294   281.13171105
   -666.03083313   180.25290246]
 [ 8480.52626497 -1927.10761294 15581.48412256 -1431.2727512
   5628.41626799  -952.07950107]
 [ -809.8455958    281.13171105 -1431.2727512    202.23251075
   -497.06830607   130.05541107]
 [ 3047.01734202  -666.03083313  5628.41626799  -497.06830607
   2043.8421743   -332.31437126]
 [ -536.10254139   180.25290246  -952.07950107   130.05541107
   -332.31437126    83.90897737]]
Trace: 22944.114440696645
Determinant: 14.250379080265544
Condition number: 275012605.4458762
Minimal eigen value: 8.242855247395328e-05
Eigen values: [2.26688910e+04 2

pressure: 878595.5231495928
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 4.76837158203125e-06
Solve time with direct kaug mode [s]: 0.4627718925476074
Total wall clock time [s]: 0.46361517906188965
Existed information has been added.
FIM: [[ 4703.93933682 -1206.80109065  8583.8191108   -886.31436223
   3080.03178823  -582.92183426]
 [-1206.80109065   499.36590521 -2109.05526834   354.94802696
   -724.1854787    225.44748474]
 [ 8583.8191108  -2109.05526834 15750.30957157 -1556.2571859
   5682.37623647 -1028.60333069]
 [ -886.31436223   354.94802696 -1556.2571859    252.93801175
   -537.01619925   161.10022694]
 [ 3080.03178823  -724.1854787   5682.37623647  -537.01619925
   2061.08885253  -356.7731431 ]
 [ -582.92183426   225.44748474 -1028.60333069   161.10022694
   -356.7731431    102.91639418]]
Trace: 23370.55807205506
Determinant: 21.59925875799877
Condition number: 267064679.0217687
Minimal eigen value: 8.611035840252282e-05
Eigen values: [2.29970352e+04 3.5

...

pressure: 580997.6629455714
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.0067901611328125e-06
Solve time with direct kaug mode [s]: 0.4943680763244629
Total wall clock time [s]: 0.4951174259185791
Existed information has been added.
FIM: [[ 4662.03143668 -1083.56909274  8581.71596093  -802.71618432
   3108.82970305  -532.63151558]
 [-1083.56909274   389.03067855 -1910.148353     279.19719062
   -661.72428841   179.10241807]
 [ 8581.71596093 -1910.148353   15897.57301056 -1422.01203102
   5796.33697048  -948.43820369]
 [ -802.71618432   279.19719062 -1422.01203102   200.94173914
   -495.17010825   129.29742579]
 [ 3108.82970305  -661.72428841  5796.33697048  -495.17010825
   2127.20479444  -332.05245537]
 [ -532.63151558   179.10241807  -948.43820369   129.29742579
   -332.05245537    83.47230337]]
Trace: 23360.25396274071
Determinant: 113.34337103572993
Condition number: 152948362.00781482
Minimal eigen value: 0.00015081984835536326
Eigen values: [2.30676488e+0

pressure: 5276042.8871137025
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 4.5299530029296875e-06
Solve time with direct kaug mode [s]: 0.4866821765899658
Total wall clock time [s]: 0.4882051944732666
Existed information has been added.
FIM: [[15426.23433541 -4500.45948243 33012.68874197 -4062.1988167
  13952.23646952 -3304.02119079]
 [-4500.45948243  1465.56511369 -9665.29845526  1306.13735979
  -4103.76081086  1052.26206525]
 [33012.68874197 -9665.29845526 71347.32963546 -8819.90139501
  30407.07480486 -7238.52669803]
 [-4062.1988167   1306.13735979 -8819.90139501  1180.57237287
  -3778.64045326   962.23196676]
 [13952.23646952 -4103.76081086 30407.07480486 -3778.64045326
  13050.3982147  -3123.8377909 ]
 [-3304.02119079  1052.26206525 -7238.52669803   962.23196676
  -3123.8377909    791.67795309]]
Trace: 103261.77762521889
Determinant: 61803.22414809353
Condition number: 481179295.102437
Minimal eigen value: 0.00021348889240148527
Eigen values: [1.02726435e+05 

pressure: 944740938.5156273
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.0067901611328125e-06
Solve time with direct kaug mode [s]: 0.48601269721984863
Total wall clock time [s]: 0.4870932102203369
Existed information has been added.
FIM: [[  874841.17938427  -946777.77383331  1983558.92935111  -902955.00122374
    879668.75538803  -767611.95969096]
 [ -946777.77383331  1028036.59934837 -2148277.97225098   980611.52246615
   -953290.45669953   833733.11657446]
 [ 1983558.92935111 -2148277.97225098  4498348.11507042 -2048966.16230581
   1995255.50003917 -1741928.11231065]
 [ -902955.00122374   980611.52246615 -2048966.16230581   935396.52559479
   -909262.70133707   795304.57915997]
 [  879668.75538803  -953290.45669953  1995255.50003917  -909262.70133707
    885114.69665319  -773036.17224002]
 [ -767611.95969096   833733.11657446 -1741928.11231065   795304.57915997
   -773036.17224002   676202.7136647 ]]
Trace: 8897939.829715746
Determinant: 87451555.9189625
Co

pressure: 3823796.9651398356
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.0067901611328125e-06
Solve time with direct kaug mode [s]: 0.46774792671203613
Total wall clock time [s]: 0.4684140682220459
Existed information has been added.
FIM: [[ 9243.66848828 -2099.87522835 19169.93403936 -1789.51507723
   7893.89756759 -1386.64255891]
 [-2099.87522835   611.00977376 -4258.06958815   494.71865243
  -1722.4621977    365.61158003]
 [19169.93403936 -4258.06958815 40366.89840499 -3701.76130874
  16854.53083851 -2921.41231612]
 [-1789.51507723   494.71865243 -3701.76130874   410.19338011
  -1525.10819813   310.38073486]
 [ 7893.89756759 -1722.4621977  16854.53083851 -1525.10819813
   7124.59702307 -1223.39553359]
 [-1386.64255891   365.61158003 -2921.41231612   310.38073486
  -1223.39553359   240.17911572]]
Trace: 57996.54618592483
Determinant: 14567.192493224258
Condition number: 415269620.0027822
Minimal eigen value: 0.0001385060592295871
Eigen values: [5.75173586e+0

pressure: 950580214.8693202
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.245208740234375e-06
Solve time with direct kaug mode [s]: 0.47016048431396484
Total wall clock time [s]: 0.470813512802124
Existed information has been added.
FIM: [[ 2601289.44876555 -1720203.28236988  6006884.73188146 -1669982.07276594
   2712759.24551402 -1445078.00976922]
 [-1720203.28236988  1138506.43438032 -3973136.08569047  1105334.29316162
  -1794606.94504609   956518.61303716]
 [ 6006884.73188146 -3973136.08569047 13872193.77934062 -3857283.99700029
   6265186.40570936 -3337898.16956506]
 [-1669982.07276594  1105334.29316162 -3857283.99700029  1073154.1898884
  -1742326.35372051   928687.14757143]
 [ 2712759.24551402 -1794606.94504609  6265186.40570936 -1742326.35372051
   2829720.01804384 -1507751.95460821]
 [-1445078.00976922   956518.61303716 -3337898.16956506   928687.14757143
  -1507751.95460821   803678.3004259 ]]
Trace: 22318542.170844633
Determinant: 115412694.31098141
Co

In [8]:
fixed = {}

all_fim.extract_criteria()
print(all_fim.store_all_results_dataframe)

all_fim.figure_drawing(fixed, ['Temperature','x_R32'], 'PR quadratic','Temperature [K]', 'x (R32)' )


Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.2)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.4)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.5)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.6)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.7)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.8)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.9)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.2)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (283, 0.4)
Design variable:  ['Temperature', 'x_R32']
Value          :  (28